In [1]:
import cv2
import numpy as np
import glob
import math

In [2]:
def find_centroid_and_angle(binary_image):
    moments = cv2.moments(binary_image)
    centroid_x = int(moments['m10'] / moments['m00'])
    centroid_y = int(moments['m01'] / moments['m00'])
    
    # Calculating orientation (angle of principal axis)
    angle = 0.5 * math.atan2(2 * moments['mu11'], (moments['mu20'] - moments['mu02']))
    angle_degrees = math.degrees(angle)
    return (centroid_x, centroid_y), angle_degrees

def crop_to_bounding_box(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
        return image[y:y+h, x:x+w]
    return image

def rotate_image(image, angle, centroid):
    (h, w) = image.shape[:2]
    M = cv2.getRotationMatrix2D(centroid, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

def reverse_image_if_needed(image, centroid, angle):
    _, width = image.shape
    line_end_x = int(centroid[0] + width * math.cos(math.radians(angle + 90)))
    line_end_y = int(centroid[1] + width * math.sin(math.radians(angle + 90)))

    # Check lengths above and below the principal axis
    length_above = centroid[1]
    length_below = image.shape[0] - centroid[1]

    if length_below > length_above:
        return cv2.flip(image, 1) # flip horizontally
    return image

def process_image(file_path):
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 1, 255, cv2.THRESH_BINARY)

    cropped_image = crop_to_bounding_box(binary_image)
    centroid, angle = find_centroid_and_angle(cropped_image)
    rotated = rotate_image(cropped_image, -angle, centroid)
    final_image = reverse_image_if_needed(rotated, centroid, angle)

    return final_image

In [3]:
for file_path in glob.glob('*.png'):
    processed_image = process_image(file_path)
    cv2.imshow('Processed Image', processed_image)
    cv2.waitKey(0) # Wait for a key press to close the image window
    cv2.destroyAllWindows()